In [12]:
# importing required libraries

from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd
import cv2

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from scipy.misc import imresize
from sklearn.metrics import log_loss

import numpy as np

In [2]:
from keras.datasets import mnist
from keras import backend as K
# load and pre-process data

if K.image_data_format() == 'channels_first':
    input_shape = (1, 28, 28)  # image shape
else:
    input_shape = (28, 28, 1)  # image shape
    
def preprocess_input(x):
    return x.astype('float32').reshape((-1,) + input_shape) / 255


def preprocess_output(y):
    return keras.utils.to_categorical(y)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = map(preprocess_input, [x_train, x_test])
y_train, y_test = map(preprocess_output, [y_train, y_test])
print('Loading MNIST data...')
print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape, 'y_test shape', y_test.shape)

Loading MNIST data...
x_train shape: (60000, 28, 28, 1) y_train shape: (60000, 10)
x_test shape: (10000, 28, 28, 1) y_test shape (10000, 10)


In [3]:
from PIL import Image

# preparing the train dataset
train_img=[]
#for i in range(len(train)):
for i in range(3000):
    temp_img = Image.new("L", (28, 28))
    temp_img.putdata(x_train[i])
    temp_img = temp_img.resize((224,224)).convert('RGB')
    temp_img = image.img_to_array(temp_img)
    train_img.append(temp_img)
    


In [4]:

#converting train images to array and applying mean subtraction processing

train_img=np.array(train_img) 
#train_img=preprocess_input(train_img)


In [5]:
# preparing the test dataset
test_img=[]
#for i in range(len(test)):
for i in range(200):
    temp_img = Image.new("L", (28, 28))
    temp_img.putdata(x_test[i])
    temp_img = temp_img.resize((224,224)).convert('RGB')
    temp_img = image.img_to_array(temp_img)
    test_img.append(temp_img)

In [6]:
#converting test images to array and applying mean subtraction processing

test_img=np.array(test_img) 
#test_img=preprocess_input(test_img)

In [7]:
from keras.models import Model


def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):
    model = VGG16(weights='imagenet', include_top=True)
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    x=Dense(num_classes, activation='softmax')(model.output)
    model=Model(model.input,x)
    #To set the first 8 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:8]:
       layer.trainable = False
    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
train_y=np.array(y_train[0:3000])
# creating training and validation set


from sklearn.model_selection import train_test_split
#X_train, X_valid, Y_train, Y_valid=train_test_split(train_x,train_y,test_size=0.3, random_state=42)

X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

In [14]:
# Example to fine-tune on 3000 samples from Cifar10
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 10 
batch_size = 16 
#nb_epoch = 10
nb_epoch = 1
# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
# Start Fine-tuning
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))



Train on 2400 samples, validate on 600 samples
Epoch 1/1
2400/2400 [==============================] - 853s 356ms/step - loss: 2.3023 - acc: 0.1079 - val_loss: 2.3021 - val_acc: 0.1017


In [16]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)

600/600 [==============================] - 75s 125ms/step


In [17]:
print(score)

2.3021375604470573
